In [18]:
import json
from langchain_core.documents import Document
from backend.app.database.log_query import get_query_logger

In [19]:
with open('/Users/yuyangyang/Documents/Studying/CUHK/9_IS_Practicum/hkia_saka_v2/backend/process_image_20250507_120228.json', 'r', encoding='utf-8') as f:
    img_content = json.load(f)
    
with open('/Users/yuyangyang/Documents/Studying/CUHK/9_IS_Practicum/hkia_saka_v2/backend/process_text_20250507_120228.json', 'r', encoding='utf-8') as f:
    text_content = json.load(f)

docs1 = img_content + text_content
docs = [Document(page_content=item["page_content"], metadata=item["metadata"]) for item in docs1]

In [21]:
import random
from datetime import datetime, timedelta

def generate_random_recent_date(days_back=90):
    """
    Generate a random date within the specified number of days from now
    
    Args:
        days_back (int): Maximum number of days in the past
        
    Returns:
        str: ISO formatted date string
    """
    # Get current date and time
    now = datetime.now()
    
    # Generate random days back (from 0 to days_back)
    random_days = random.randint(0, days_back)
    
    # Generate random hours, minutes, seconds and microseconds
    random_hour = random.randint(0, 23)
    random_minute = random.randint(0, 59)
    random_second = random.randint(0, 59)
    random_microsecond = random.randint(0, 999999)
    
    # Calculate random date
    random_date = now - timedelta(days=random_days)
    
    # Set random time components
    random_datetime = random_date.replace(
        hour=random_hour,
        minute=random_minute, 
        second=random_second,
        microsecond=random_microsecond
    )
    
    # Return ISO formatted string
    return random_datetime.isoformat()

# Example: Generate a single random date
random_date = generate_random_recent_date()
print(random_date)

2025-03-26T02:25:27.429511


In [17]:
logger = get_query_logger()

In [26]:
import sqlite3

# Generate 100 random logs spanning the past 90 days
for _ in range(200):
    # Get random timestamp
    timestamp = generate_random_recent_date()
    
    # Get random docs (select 2-5 random docs from the collection)
    num_docs = random.randint(1, 3)
    random_docs = random.sample(docs, num_docs)
    
    # Log to database
    logger.log_documents(random_docs)
    
    # Update the timestamp directly in the database
    conn = sqlite3.connect('backend/data/logs/query_logs.db')
    cursor = conn.cursor()
    # 获取刚刚插入的记录IDs
    cursor.execute("SELECT id FROM doc_logs ORDER BY id DESC LIMIT ?", (num_docs,))
    recent_ids = [row[0] for row in cursor.fetchall()]
    
    # 更新每个记录的时间戳
    for doc_id in recent_ids:
        cursor.execute("UPDATE doc_logs SET timestamp = ? WHERE id = ?", (timestamp, doc_id))
    
    conn.commit()
    conn.close()